In [1]:
from google.auth.transport import requests
from googleapiclient import discovery
from google.oauth2 import service_account
from google.cloud import storage
from googleapiclient.http import MediaIoBaseDownload
import io
import threading
from time import sleep
import os
import pandas as pd

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive, GoogleDriveFile
from oauth2client.service_account import ServiceAccountCredentials

from datetime import time, datetime, timedelta

drivers_check_list = []
drivers_checked_list = []
ls_threads = []
ls_tmp = []
ls_error = []
ls_error_files = []
ls_files_to_download = []
ls_files_to_download_tmp = []
count_files_checked = 0
count_files_down = 0
count_files_error = 0
file_down_parms = []
status = None

drivers = [
            ['Ci', '0AHj-PRcbOxSWUk9PVA'],
           ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
           ['VG.COM', '0AElRg0lTw8iIUk9PVA'],
           ['Архив', '0AOP_oKFy4oGJUk9PVA'],
           ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
           ['Контент_VG', '0AN6chrg4ApnnUk9PVA'],
           ['Контент', '0ANf3LpamTrsTUk9PVA'],
           ['Лобня', '0AIQWy4IyyeaIUk9PVA'],
           ['Объединенная пивоварня', '0AELFNnmNadJqUk9PVA'],
           ['ОСК Шереметьево', '0ALRUo2DlF1YmUk9PVA'],
           ['Мой Диск _', '0AHpbUawYyiEZUk9PVA'],
    ['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объекты в работе', '0AOxaPQRooOk2Uk9PVA'],
 ['Остров мечты ППР', '0AImPD4jQdMXJUk9PVA'],
 ['Пекарня', '0AIWIhafOWgmHUk9PVA'],
 ['ПОДРЯДЧИКИ', '0ALnZgMLQFfrtUk9PVA'],
 ['СТО ГБО', '0AF2NtBbx_XvYUk9PVA'],
 ['ТЕХВАГОНМАШ', '0AAAyvFQMbstZUk9PVA'],
 ['Транспроект', '0ADKHIbPu575TUk9PVA'],
 ['ФосАгро', '0AD-4c3pLaBV5Uk9PVA'],
 ['ФРАКДЖЕТ-ТУЛЗ', '0ACjAWTbTqvESUk9PVA'],
 ['Холодный склад', '0ALT2Y1wkG7vCUk9PVA'],
 ['Чайка Лаб', '0APbGeMNQtN3MUk9PVA'],
 ['Читинские ключи', '0AA9gFHffXEitUk9PVA'],
 ['Юрист UA', '0AGg7JoJwa_8nUk9PVA']
           ]

storage.blob._DEFAULT_CHUNKSIZE = 10 * 1024* 1024  # 5 MB
storage.blob._MAX_MULTIPART_SIZE = 5 * 1024* 1024  # 5 MB


class GDrive:
    def __init__(self,  drive_name, drive_id, get_files_folders=True, get_files_fullname=True):
        self.drive_name = drive_name
        self.drive_id = drive_id

        self.drive = None
        self.service = None
        self.bucket = None

        self.get_client()

        self.bucet_main_folder = 'GoogleDriveTransfer'

        self.dict_files = {}
        self.dict_files_fullname = {}
        self.dict_folders = {}

        if get_files_folders:
            self.get_files_folders()
            if get_files_fullname:
                self.get_files_fullname()

        blobs = self.bucket.list_blobs(prefix=f'GoogleDriveTransfer/{drive_name}')
        self.files_blobs = [blob.name for blob in blobs]

    def get_client(self):
        # Google Drive
        gauth = GoogleAuth()
        scope = ["https://www.googleapis.com/auth/drive"]
        gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('savvy-concord-374520-bbbb0e3c73c3.json', scope)
        self.drive = GoogleDrive(gauth)

        credentials_drive = service_account.Credentials.from_service_account_file(
            filename='savvy-concord-374520-bbbb0e3c73c3.json',
            scopes=['https://www.googleapis.com/auth/drive'],
        )
        session = requests.AuthorizedSession(credentials_drive)
        self.service = discovery.build('drive', 'v3', credentials=credentials_drive)

        #Google Storage
        credentials_storage = service_account.Credentials.from_service_account_file(
            filename='credentials.json',
            scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
        )
        client = storage.Client(credentials=credentials_storage)
        self.bucket = client.get_bucket('cabinet_bucket')


    def get_files_folders(self):
        self.get_files()
        self.get_folders()

    def get_folders(self):
        self.dict_folders = {}
        ls_folders = self.drive.ListFile(
            {
                'q': "mimeType='application/vnd.google-apps.folder'",
                'corpora': 'drive',
                'driveId': self.drive_id,
                'includeItemsFromAllDrives': True,
                'supportsAllDrives': True,
            }
        ).GetList()
        for folder in ls_folders:
            self.dict_folders[folder['id']] = folder

    def get_files(self):
        self.dict_files = {}
        ls_files = self.drive.ListFile(
            {
                'q': "mimeType!='application/vnd.google-apps.folder'",
                'corpora': 'drive',
                'driveId': self.drive_id,
                'includeItemsFromAllDrives': True,
                'supportsAllDrives': True,
            }
        ).GetList()
        for file in ls_files:
            self.dict_files[file['id']] = file

    def get_files_fullname(self):
        self.dict_files_fullname = {}
        for file in self.dict_files.values():
            parent_id = file['parents'][0]['id']
            name_full = self.get_full_path(file['title'], parent_id)
            self.dict_files_fullname[file['id']] = name_full

    def get_full_path(self, file_name, parent_id):
        tree = [file_name]
        folder_id = parent_id

        while folder_id != self.drive_id:
            folder = self.dict_folders[folder_id]
            tree.append(folder['title'])
            folder_id = folder['parents'][0]['id']

        tree.append(self.drive_name)
        tree.append(self.bucet_main_folder)

        tree.reverse()
        return '/'.join(tree)

    def get_files_to_download(self):
        for file_id in self.dict_files.keys():
            par = self.get_params_down(file_id)
            if not par['is_propusk'] :
                if par['fullname'] not in self.files_blobs:
                    gdfile = self.dict_files[file_id]
                    size = int(gdfile['fileSize']) / 1024 / 1024
                    if size < 3000:
                        ls_files_to_download.append([file_id,  par['fullname']])
                        ls_files_to_download_tmp.append([file_id,  par['fullname'], size])


    def get_params_down(self, file_id):
        gdfile = self.dict_files[file_id]
        mimeType = gdfile['mimeType']
        namefile = gdfile['title']
        fullname = self.dict_files_fullname[file_id]

        is_google_apps = False
        is_propusk = False

        ls_mime_google_apps = ['application/vnd.google-apps.document', 'application/vnd.google-apps.spreadsheet',
                               'application/vnd.google-apps.presentation', 'application/vnd.google-apps.site',
                               'application/vnd.google-apps.shortcut']

        if mimeType in ls_mime_google_apps:
            is_propusk = True


        res = {
            'namefile': namefile,
            'fullname': fullname,
            'mimeType': mimeType,
            'is_google_apps': is_google_apps,
            'is_propusk': is_propusk,
        }

        return res

In [2]:
def f12():
    for d in drivers:
        driv = GDrive(d[0], d[1])
        driv.get_files_to_download()

t = threading.Thread(target=f12)
t.start()

In [2]:


# текущее время
now = datetime.now()
now = now.strftime("%d/%m/%Y %H:%M:%S")
now

'19/01/2023 18:43:28'

In [2]:
ls_files_to_download1 = []
ls_files_to_download_check = []
def get_client():
    # Google Drive
    gauth = GoogleAuth()
    scope = ["https://www.googleapis.com/auth/drive"]
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('savvy-concord-374520-bbbb0e3c73c3.json', scope)
    drive = GoogleDrive(gauth)

    credentials_drive = service_account.Credentials.from_service_account_file(
        filename='savvy-concord-374520-bbbb0e3c73c3.json',
        scopes=['https://www.googleapis.com/auth/drive'],
    )
    session = requests.AuthorizedSession(credentials_drive)
    service = discovery.build('drive', 'v3', credentials=credentials_drive)

    #Google Storage
    credentials_storage = service_account.Credentials.from_service_account_file(
        filename='credentials.json',
        scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
    )
    client = storage.Client(credentials=credentials_storage)
    bucket = client.get_bucket('cabinet_bucket')

    return service, drive, bucket


def download_files(ls_files_to_download2):
    global count_files_checked
    is_download_files = True

    service, drive, bucket = get_client()

    ls_files_to_download1.clear()

    for file_id, fullname in ls_files_to_download2:
        count_files_checked += 1
        done = download_file(file_id, fullname, service, bucket)
        if not done:
            ls_files_to_download1.append([file_id, fullname])
            is_download_files = False
        else:
            ls_files_to_download_check.append([file_id, fullname])

    return is_download_files


def download_file(file_id, fullname, service, bucket):
    is_download = True
    global count_files_down
    try:
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()

        blob = bucket.blob(fullname)
        if not blob.exists():
            blob.upload_from_string(fh.getvalue(), timeout=300)
            count_files_down += 1
        is_download = True

    except Exception as e:
        ls_error.append([file_id, fullname, 'Ошибка', e])
        is_download = False
    finally:
        fh.close()
    return is_download

def transfer_drive():
    ls = ls_files_to_download
    while True:
        done = download_files(ls)

        # текущее время
        now = datetime.now()
        now = now.strftime("%d/%m/%Y %H:%M:%S")
        ls_tmp.append(f'Завершено------------------------{done}__ {now}')

        if done:
            break

        ls = ls_files_to_download1

        sleep(60)

def transfer_drives_thread():
    t = threading.Thread(target=transfer_drive)
    ls_threads.append(t)
    t.start()

    return t

In [3]:
df1 = pd.read_csv('files_to_download.csv')
ls_files_to_download = df1.values.tolist()

t = transfer_drives_thread()

In [7]:
df1 = pd.read_csv('files_to_download.csv')
ls_files_to_download = df1.values.tolist()

In [11]:
ls3 = df1.values.tolist()

In [5]:
df = pd.DataFrame(ls_files_to_download, columns=['file_id', 'fullname'])
df

,file_id,fullname
0,1kbkox1Q2F2mmLLLRDmJNN8Vqo32HziRv,GoogleDriveTransfer/Ci/Выпуск/mt45mssql/mt45ms...
1,1YaFZXs19Vt56KLlPRAJkaHWkw7Wxcw_x,GoogleDriveTransfer/Ci/Выпуск/mt45mssql/mt45ms...
2,1PeV17i0gFFPA_u6r3SaMnSmJbMU4C4V8,GoogleDriveTransfer/Ci/mt45mssql.src/mt45mssql...
3,1VwI--l2Gwm9BmHXOPRWLvgeH6wWCmF5m,GoogleDriveTransfer/Ci/Опытное тестирование De...
4,1NqyQk0pJt0Qh_FH0dtBJPIHDIuMqM0MF,GoogleDriveTransfer/Ci/MT5/v2/Libraries/MsSql_...
...,...,...
151,130siwH65a-lQESNb9fBHHGbbElbWsadB,GoogleDriveTransfer/ПОДРЯДЧИКИ/1 Подрядчики ПЛ...
152,1WPAjnS5to81VeRCOi-Rkte0OhoRYTYrA,GoogleDriveTransfer/Транспроект/Архив/Вагонмаш...
153,13CKZJ5pBV3CZOjUn23z7VaKK4cnK98YL,GoogleDriveTransfer/Транспроект/Архив/Попаснян...
154,1hrfZtzCh4jLvLnFeMGqnSiz4oIdjKd5q,GoogleDriveTransfer/Транспроект/Архив/Вагонмаш...


In [6]:
df.to_csv('files_to_download.csv', index=False)